In [ ]:
import numpy as np
import random
from dataset import atti_dirigenti

from keras import layers, models, optimizers, utils, metrics
from keras.callbacks import *
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from scipy import stats

In [ ]:
(x_train, y_train),(x_val, y_val), (x_test, y_test) = atti_dirigenti.load_data(num_words=5000, remove_stopwords=True)

In [ ]:
x_train[:5]

In [ ]:
label_index = atti_dirigenti.get_labels()
len(label_index)

In [ ]:
num_classes = len(label_index)

In [ ]:
def max_index(data):
    return max(data.max())

In [ ]:
max_idx = max([max_index(x_train), max_index(x_val), max_index(x_test)]) + 10

In [ ]:
max_idx

In [ ]:
num_features = max_idx 

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [ ]:
def vectorize_sequences_generator(sequences, dimension, batch_size):
    num_batches = len(sequences) // batch_size
    
    i=0
    while True:
        # to be sure don't go over the size of the dataset
        n = i % num_batches
        i +=1
        if (n+1) * batch_size < len(sequences):
            yield vectorize_sequences(sequences[n*batch_size : (n+1)*batch_size], dimension)
        else:
            yield vectorize_sequences(sequences[n*batch_size : len(sequences)], dimension)

#### Evaluate if works

In [ ]:
batch_size = 256
steps_per_epoch = len(x_train) // batch_size

i = 0
for batch in vectorize_sequences_generator(x_train, num_features, batch_size):
    print(np.argmax(batch[-1]))
    i+=1
    
    if i == 10:
        break
    

In [ ]:
batch_size = 256

In [ ]:
def to_one_hot(labels, num_classes):
    results = np.zeros((len(labels), num_classes), dtype=np.float16)
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

In [ ]:
def to_one_hot_generator(labels, batch_size, num_classes):
    num_batches = len(labels) // batch_size
    num_classes = len(set(labels))
    
    i = 0
    while True:
        n = i % num_batches
        i += 1
        if (n+1) * batch_size < len(labels):
            yield to_one_hot(labels[n*batch_size : (n+1)*batch_size], num_classes)
        else:
            yield to_one_hot(labels[n*batch_size : len(labels)], num_classes)
        

In [ ]:
batch_size = 256
steps_per_epoch = len(y_train) // batch_size

i = 0
for batch in to_one_hot_generator(y_train, batch_size, num_classes):
    for v in batch[:10]:
        print(np.argmax(v))
    break

    

In [ ]:
y_train[:10]

In [ ]:
def data_generator(data, labels, batch_size, num_features, num_classes):
    x_generator = vectorize_sequences_generator(data, num_features, batch_size)
    y_generator = to_one_hot_generator(labels, batch_size, num_classes)
    
    while True:
        yield next(x_generator), next(y_generator)

In [ ]:
for x_batch, y_batch in data_generator(x_train, y_train, 256, num_features, num_classes):
    print(x_batch.shape, y_batch.shape)
    break

In [ ]:
train_generator = data_generator(x_train, y_train, batch_size, num_features, num_classes)
val_generator = data_generator(x_val, y_val, batch_size, num_features, num_classes)
test_generator = data_generator(x_test, y_test, batch_size, num_features, num_classes)

### Dataset for hyper parameters optimization

In [ ]:
size = int(len(x_train) * 0.4)

In [ ]:
x_train_grid = vectorize_sequences(x_train[0:size], num_features)
y_train_grid = to_one_hot(y_train[0:size], num_classes)

#### Evaluate if works

### Build the Model

In [ ]:
def build_model(neurons, dropout, num_features, num_classes, activation='relu', init_mode='glorot_uniform'):
    input_tensor = layers.Input(shape=(num_features,))
    first_layer = True
    for n in neurons:
        if first_layer:
            first_layer = False
            l = layers.Dense(n, activation=activation, kernel_initializer=init_mode)(input_tensor)
            l = layers.Dropout(dropout)(l)
        else:
            l_next = layers.Dense(n, activation=activation, kernel_initializer=init_mode)(l)
            l_next = layers.Dropout(dropout)(l_next)
            l = l_next
    last_layer = layers.Dense(num_classes, activation='softmax')(l)
    model = models.Model(inputs = [input_tensor], outputs = [last_layer])
    
    model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics=['accuracy'])
    return model

### Search for the best parameters

In [ ]:
wrapped_model = KerasClassifier(build_fn=build_model, num_features=num_features, num_classes=num_classes)

In [ ]:
# neurons = sorted(np.random.randint(0,128,(5,1)))
# dropout = sorted(np.random.rand(5))
base = [16,64,128]
neurons = [[x] for x in base]
neurons += [[x,x] for x in base]
neurons += [[x,x,x] for x in base]
dropout = np.arange(0.1, 0.7, step=0.1)
activation = ['relu', 'tanh', 'sigmoid']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'glorot_normal']

In [ ]:
param_dist = dict(neurons=neurons, dropout=dropout, activation=activation)

In [ ]:
grid = GridSearchCV(wrapped_model, param_grid=param_dist)

In [ ]:
grid_result= grid.fit(X=x_train_grid, y=y_train_grid)

### Summarize Results

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
results = """
Best: 0.744743 using {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [128, 128]}
0.647776 (0.008467) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [16]}
0.726733 (0.004107) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [64]}
0.741926 (0.002003) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [128]}
0.634923 (0.003979) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [16, 16]}
0.729450 (0.002776) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [64, 64]}
0.742605 (0.001667) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [128, 128]}
0.597746 (0.008815) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [16, 16, 16]}
0.718005 (0.001712) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [64, 64, 64]}
0.734254 (0.006787) with: {'activation': 'relu', 'dropout': 0.10000000000000001, 'neurons': [128, 128, 128]}
0.635904 (0.007324) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [16]}
0.720948 (0.001441) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [64]}
0.737272 (0.003113) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [128]}
0.600186 (0.010220) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [16, 16]}
0.716294 (0.005748) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [64, 64]}
0.739410 (0.001635) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [128, 128]}
0.575485 (0.014497) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [16, 16, 16]}
0.702108 (0.001849) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [64, 64, 64]}
0.733927 (0.001513) with: {'activation': 'relu', 'dropout': 0.20000000000000001, 'neurons': [128, 128, 128]}
0.630471 (0.005090) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [16]}
0.717099 (0.000895) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [64]}
0.732619 (0.001035) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [128]}
0.591181 (0.006098) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [16, 16]}
0.704573 (0.005510) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [64, 64]}
0.729852 (0.000847) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [128, 128]}
0.437418 (0.025241) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [16, 16, 16]}
0.669484 (0.001294) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [64, 64, 64]}
0.719212 (0.000869) with: {'activation': 'relu', 'dropout': 0.30000000000000004, 'neurons': [128, 128, 128]}
0.612713 (0.003327) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [16]}
0.703315 (0.004689) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [64]}
0.728343 (0.002462) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [128]}
0.524525 (0.010194) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [16, 16]}
0.685205 (0.006098) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [64, 64]}
0.719388 (0.002806) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [128, 128]}
0.382860 (0.025280) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [16, 16, 16]}
0.624057 (0.017902) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [64, 64, 64]}
0.693078 (0.006377) with: {'activation': 'relu', 'dropout': 0.40000000000000002, 'neurons': [128, 128, 128]}
0.594174 (0.007287) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [16]}
0.697127 (0.002701) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [64]}
0.721426 (0.004081) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [128]}
0.439305 (0.045147) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [16, 16]}
0.644984 (0.003787) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [64, 64]}
0.702862 (0.000730) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [128, 128]}
0.292585 (0.028300) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [16, 16, 16]}
0.569625 (0.008936) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [64, 64, 64]}
0.641186 (0.008019) with: {'activation': 'relu', 'dropout': 0.5, 'neurons': [128, 128, 128]}
0.571763 (0.002188) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [16]}
0.681004 (0.002096) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [64]}
0.710937 (0.002770) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [128]}
0.392872 (0.009480) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [16, 16]}
0.599557 (0.005861) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [64, 64]}
0.665459 (0.006254) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [128, 128]}
0.243661 (0.034925) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [16, 16, 16]}
0.434400 (0.029201) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [64, 64, 64]}
0.576492 (0.001909) with: {'activation': 'relu', 'dropout': 0.59999999999999998, 'neurons': [128, 128, 128]}
0.666642 (0.005959) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [16]}
0.734606 (0.002003) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [64]}
0.742504 (0.001739) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [128]}
0.666918 (0.009338) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [16, 16]}
0.741498 (0.002034) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [64, 64]}
0.742504 (0.001951) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [128, 128]}
0.659297 (0.006097) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [16, 16, 16]}
0.737876 (0.001760) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [64, 64, 64]}
0.741020 (0.001611) with: {'activation': 'tanh', 'dropout': 0.10000000000000001, 'neurons': [128, 128, 128]}
0.655901 (0.004159) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [16]}
0.728670 (0.001278) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [64]}
0.740995 (0.002773) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [128]}
0.661812 (0.005461) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [16, 16]}
0.737448 (0.002064) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [64, 64]}
0.744743 (0.000188) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [128, 128]}
0.628534 (0.004568) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [16, 16, 16]}
0.728947 (0.003336) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [64, 64, 64]}
0.736367 (0.003389) with: {'activation': 'tanh', 'dropout': 0.20000000000000001, 'neurons': [128, 128, 128]}
0.647802 (0.007143) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [16]}
0.726658 (0.000410) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [64]}
0.739184 (0.003619) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [128]}
0.635954 (0.004607) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [16, 16]}
0.732317 (0.003016) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [64, 64]}
0.739737 (0.004068) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [128, 128]}
0.598023 (0.006738) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [16, 16, 16]}
0.719338 (0.004305) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [64, 64, 64]}
0.730305 (0.002373) with: {'activation': 'tanh', 'dropout': 0.30000000000000004, 'neurons': [128, 128, 128]}
0.637086 (0.002333) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [16]}
0.716974 (0.002936) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [64]}
0.738530 (0.003610) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [128]}
0.600513 (0.005159) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [16, 16]}
0.723237 (0.003644) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [64, 64]}
0.735084 (0.001356) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [128, 128]}
0.531769 (0.018333) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [16, 16, 16]}
0.707390 (0.003889) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [64, 64, 64]}
0.722759 (0.003134) with: {'activation': 'tanh', 'dropout': 0.40000000000000002, 'neurons': [128, 128, 128]}
0.631150 (0.003134) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [16]}
0.711289 (0.001154) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [64]}
0.731110 (0.002615) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [128]}
0.571687 (0.005546) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [16, 16]}
0.708799 (0.001763) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [64, 64]}
0.731688 (0.000094) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [128, 128]}
0.493661 (0.011795) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [16, 16, 16]}
0.678288 (0.001322) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [64, 64, 64]}
0.715993 (0.003346) with: {'activation': 'tanh', 'dropout': 0.5, 'neurons': [128, 128, 128]}
0.609116 (0.001555) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [16]}
0.702158 (0.004177) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [64]}
0.724847 (0.002343) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [128]}
0.523619 (0.005351) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [16, 16]}
0.684048 (0.004778) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [64, 64]}
0.722306 (0.003777) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [128, 128]}
0.417748 (0.035234) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [16, 16, 16]}
0.621214 (0.006650) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [64, 64, 64]}
0.689758 (0.000996) with: {'activation': 'tanh', 'dropout': 0.59999999999999998, 'neurons': [128, 128, 128]}
0.490090 (0.006018) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [16]}
0.600563 (0.004491) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [64]}
0.642947 (0.001345) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [128]}
0.312003 (0.037418) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [16, 16]}
0.536749 (0.023760) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [64, 64]}
0.609518 (0.006652) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [128, 128]}
0.174263 (0.021225) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [16, 16, 16]}
0.484682 (0.008603) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [64, 64, 64]}
0.570077 (0.009896) with: {'activation': 'sigmoid', 'dropout': 0.10000000000000001, 'neurons': [128, 128, 128]}
0.465339 (0.015995) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [16]}
0.592992 (0.003917) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [64]}
0.632659 (0.005712) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [128]}
0.281718 (0.018895) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [16, 16]}
0.511017 (0.009289) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [64, 64]}
0.580214 (0.005515) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [128, 128]}
0.202510 (0.013384) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [16, 16, 16]}
0.408542 (0.006377) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [64, 64, 64]}
0.517708 (0.013010) with: {'activation': 'sigmoid', 'dropout': 0.20000000000000001, 'neurons': [128, 128, 128]}
0.438424 (0.005164) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [16]}
0.580189 (0.000339) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [64]}
0.614272 (0.007204) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [128]}
0.271481 (0.017266) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [16, 16]}
0.473941 (0.010772) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [64, 64]}
0.548622 (0.004295) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [128, 128]}
0.144783 (0.029559) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [16, 16, 16]}
0.331598 (0.019725) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [64, 64, 64]}
0.479550 (0.013465) with: {'activation': 'sigmoid', 'dropout': 0.30000000000000004, 'neurons': [128, 128, 128]}
0.457214 (0.010808) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [16]}
0.561676 (0.007625) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [64]}
0.601041 (0.008791) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [128]}
0.202611 (0.026743) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [16, 16]}
0.444914 (0.021984) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [64, 64]}
0.511973 (0.012992) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [128, 128]}
0.124384 (0.018447) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [16, 16, 16]}
0.278826 (0.021667) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [64, 64, 64]}
0.345206 (0.003879) with: {'activation': 'sigmoid', 'dropout': 0.40000000000000002, 'neurons': [128, 128, 128]}
0.407159 (0.020860) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [16]}
0.545980 (0.003657) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [64]}
0.585220 (0.002003) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [128]}
0.210937 (0.003901) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [16, 16]}
0.352022 (0.014045) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [64, 64]}
0.469816 (0.004079) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [128, 128]}
0.111706 (0.016743) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [16, 16, 16]}
0.198486 (0.019712) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [64, 64, 64]}
0.292107 (0.009168) with: {'activation': 'sigmoid', 'dropout': 0.5, 'neurons': [128, 128, 128]}
0.351947 (0.015003) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [16]}
0.521456 (0.003706) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [64]}
0.557476 (0.010049) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [128]}
0.164579 (0.042435) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [16, 16]}
0.294723 (0.010880) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [64, 64]}
0.339722 (0.010352) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [128, 128]}
0.100941 (0.001581) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [16, 16, 16]}
0.102249 (0.003393) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [64, 64, 64]}
0.130647 (0.026755) with: {'activation': 'sigmoid', 'dropout': 0.59999999999999998, 'neurons': [128, 128, 128]}

"""

print(results)